In [ ]:
# !pip install sentence_transformers
# !pip install langchain

# !pip install sentence_transformers

# !pip install transformers

# !sudo apt-get install libreoffice
# !pip install pydantic==1.10.8
# !pip install chromadb

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from langchain.llms import HuggingFacePipeline
import torch
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
# from AutoClean import AutoClean

In [ ]:

# pipeline = AutoClean(df, mode = 'auto' , duplicates = 'auto', missing_num = 'auto', missing_categ = 'auto',
#         encode_categ = 'auto', extract_datetime=True, outliers='auto', outlier_param=1.5,
#         logfile=False, verbose=False)


# data = pipeline.output

In [28]:
text_splitter = CharacterTextSplitter( chunk_size = 500 , chunk_overlap = 50 )
embedding_table = SentenceTransformerEmbeddings(model_name = "deepset/all-mpnet-base-v2-table" )


def Recommender(data , exclude_column , Category_column , output_path ) :

    
    columns_to_combine = [col for col in data.columns if col not in exclude_column]
    
    data['combined_column'] = data[columns_to_combine].apply(lambda row: ' '.join(map(str, row)), axis=1)
    
    new = data[[exclude_column,Category,'combined_column']]
    
    combined_features = new[['combined_column']]
    
    combined_features.to_csv('ftest.csv')
    
    lod = CSVLoader("ftest.csv")
    
    documents = lod.load()
        
    docs = text_splitter.split_documents(documents)
    
    vectordb = Chroma.from_documents(documents=docs, embedding=embedding_table)
    
    
    new['recommended_card'] = ''

    for i in range(len(new)):            # iterate on dataframe to get Row context of combined_features column
        question = new['combined_column'][i] 
        context = vectordb.similarity_search(question) # calculate similarity between input row and rest of data

        data = []
        for document in context : #   iterate on each context of VDB to get the results and put it in a dataframe
            content = document.page_content
            rows = content.split('\n')
            row_dict = {}
            for row in rows:
                if ':' in row:
                    key, value = row.split(': ', 1)
                    row_dict[key] = value
            data.append(row_dict)
        df = pd.DataFrame(data)

        cat_list = []
        for n in df[''] : # iterate on the index to get the use_id from original dataframe
            cat_list.append(new.iloc[int(n)][Category])
            
        recommended_cards_str = ', '.join(set(cat_list))
        new.at[i, 'recommended_card'] = recommended_cards_str  # put the recommended category in the final dataframe
        
        
    os.remove("ftest.csv") 
    
    final = new[['user_id','Card_Category','recommended_card']]
    final.to_csv(output_path)
    return final

In [22]:
df = pd.read_csv('recomemndation2.csv')

In [29]:
data = df

exclude_column = 'user_id'

Category = 'Card_Category'

output_path = 'final_results.csv'

Recommender(data , exclude_column , Category , output_path   )

,user_id,Card_Category,recommended_card
0,1,Blue,Blue
1,2,Blue,Blue
2,3,Blue,Blue
3,4,Blue,Blue
4,5,Blue,Blue
...,...,...,...
10122,10123,Blue,Blue
10123,10124,Blue,Blue
10124,10125,Blue,Blue
10125,10126,Blue,Blue


In [36]:
bio = pd.read_csv('final_results.csv')
bio.head()

,Unnamed: 0,user_id,Card_Category,recommended_card
0,0,1,Blue,Blue
1,1,2,Blue,Blue
2,2,3,Blue,Blue
3,3,4,Blue,Blue
4,4,5,Blue,Blue


In [43]:
bio.loc[bio['recommended_card'] == 'Silver, Gold'].tail(15)

,Unnamed: 0,user_id,Card_Category,recommended_card
551,551,552,Gold,"Silver, Gold"
1896,1896,1897,Silver,"Silver, Gold"
3665,3665,3666,Gold,"Silver, Gold"
3989,3989,3990,Gold,"Silver, Gold"
4314,4314,4315,Silver,"Silver, Gold"
6614,6614,6615,Gold,"Silver, Gold"
8979,8979,8980,Silver,"Silver, Gold"
9044,9044,9045,Gold,"Silver, Gold"
9127,9127,9128,Platinum,"Silver, Gold"
9397,9397,9398,Gold,"Silver, Gold"


In [44]:
bio['Card_Category'].value_counts()

Blue        9436
Silver       555
Gold         116
Platinum      20
Name: Card_Category, dtype: int64

In [37]:
bio['recommended_card'].value_counts()

Blue                      9589
Silver                     429
Gold                        84
Silver, Gold                11
Blue, Silver                 4
Platinum                     3
Platinum, Silver             3
Platinum, Silver, Gold       2
Blue, Platinum               1
Blue, Silver, Gold           1
Name: recommended_card, dtype: int64

In [26]:
data = df
exclude_column = 'user_id'
Category = 'Card_Category'

Recommender(data , exclude_column , Category )

,user_id,Card_Category,combined_column,recommended_card
0,1,Blue,0 45 M 3 High School Married $60K - $80K Blue ...,Blue
1,2,Blue,1 49 F 5 Graduate Single Less than $40K Blue 4...,Blue
2,3,Blue,2 51 M 3 Graduate Married $80K - $120K Blue 36...,Blue
3,4,Blue,3 40 F 4 High School Unknown Less than $40K Bl...,Blue
4,5,Blue,4 40 M 3 Uneducated Married $60K - $80K Blue 2...,Blue
...,...,...,...,...
10122,10123,Blue,10122 50 M 2 Graduate Single $40K - $60K Blue ...,Blue
10123,10124,Blue,10123 41 M 2 Unknown Divorced $40K - $60K Blue...,Blue
10124,10125,Blue,10124 44 F 1 High School Married Less than $40...,Blue
10125,10126,Blue,10125 30 M 2 Graduate Unknown $40K - $60K Blue...,Blue
